In [1]:
%reload_ext autoreload
%autoreload 3

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import matplotlib.animation as animation
from scipy.stats import binom, t
import matplotlib as mpl
from sklearn import datasets
import itertools
from tqdm import tqdm

from utils import *

In [2]:
X_0 = np.random.rand(50,2) + 1
X_1 = np.random.rand(50,2) - 1
X = np.concatenate([X_0, X_1])

x_min, x_max = min(X[:,0]), max(X[:,0])
y_min, y_max = min(X[:,1]), max(X[:,1])

In [3]:
def build_rand_mat(K, x_min, y_min, x_max, y_max):
    X_H0 = np.random.uniform(size=(K,2)) * np.array([x_max - x_min, y_max - y_min]) + np.array([x_min, y_min])
    W, D_sqinv = compute_G0(X_H0)
    L = compute_laplacian(W, D_sqinv)
    A = compute_AM_normalised(L, D_sqinv, 3) #3 peut être faible
    W, g = compute_Ge(A)
    g.compute_A()
    g.kruskal_algo()
    mat = g.compute_mat()
    return mat, g

In [4]:
def hash_function(V_, K, N):
    return int((V_ - 1) * K / N)

In [11]:
def update_deltas(deltas, eps, K, size, x_min, y_min, x_max, y_max, update_, Q=30, conf_level=0.05):
    res_q = np.zeros((K, Q), dtype=int)
    for q in range(Q):
        res = np.zeros(K, dtype=int)
        mat, g = build_rand_mat(size - 1, x_min, y_min, x_max, y_max)
        omega = g.omega
        V = g.V
        for i in range(len(mat)):
            k = hash_function(int(mat[i, 5]), K, size) 
            pfa = compute_pfa(min(mat[i, 4] / omega, 1), V , mat[i,5])
            if pfa < deltas[k]:
                res[k] += 1
        res_q[:, q] = res.copy()
    s = np.maximum(np.std(res_q, axis=1), 1e-5)
    m = np.mean(res_q, axis=1)
    upd = t.cdf((m - eps / K) / s * np.sqrt(Q - 1), Q-1) > conf_level
    deltas -= (2 * upd - 1) * update_
    deltas = np.minimum(1, deltas)
    return deltas, res_q


In [12]:
def compute_deltas(size, Q=30, conf_level=0.05, eps=1, K=10):
    deltas = 0.99 * np.ones(K)
    update_ = 1e-3
    for i in tqdm(range(100)):
        if i == 10:
            update_ = 1e-4
        if i == 20:
            update_ = 1e-5
        if i == 30:
            update_ = 1e-6
        deltas, res_q = update_deltas(deltas, eps, K, size, x_min, y_min, x_max, y_max, update_, Q, conf_level)
        print(np.mean(np.sum(res_q, axis=0)))
    return deltas

In [13]:
compute_deltas(30, Q=30, conf_level=0.01, eps=1, K=10)

  1%|          | 1/100 [00:00<00:54,  1.82it/s]

0.7


  2%|▏         | 2/100 [00:01<00:55,  1.78it/s]

0.3333333333333333


  3%|▎         | 3/100 [00:01<00:54,  1.77it/s]

...

 99%|█████████▉| 99/100 [00:57<00:00,  1.76it/s]

0.5


100%|██████████| 100/100 [00:57<00:00,  1.73it/s]

0.4


array([0.999994, 0.999996, 0.99999 , 0.999952, 0.999822, 0.999214,
       0.996428, 0.987802, 0.97883 , 0.980886])

In [77]:
K = 10
size = 30
deltas = (1 - 1e-3) * np.ones(K)
deltas = 0.99 * np.ones(K)
eps = 1
conf_level = 0.01
update_ = 1e-3
Q = 50

prev_deltas = deltas.copy()

for i in range(501):
    if i == 10:
        update_ = 1e-4
    if i == 20:
        update_ = 1e-5
    if i == 30:
        update_ = 1e-6
    deltas, res_q = update_deltas(deltas, eps, K, size, x_min, y_min, x_max, y_max, update_, Q=Q)
    if i % 5 == 0:
        diff_delta = np.mean(np.abs(prev_deltas - deltas))
        prev_deltas = deltas.copy()
        print(i, np.mean(res_q), np.mean(np.sum(res_q, axis=0)), diff_delta)
        print(np.mean(res_q, axis=1))
        print(deltas)
print(deltas)

0 0.044 0.44 0.0010000000000000009
[0.   0.   0.   0.   0.   0.   0.   0.06 0.36 0.02]
[0.991 0.991 0.991 0.991 0.991 0.991 0.991 0.989 0.989 0.991]
5 0.022 0.22 0.004200000000000004
[0.   0.   0.   0.   0.   0.   0.   0.04 0.16 0.02]
[0.996 0.996 0.996 0.996 0.996 0.996 0.996 0.988 0.984 0.992]
10 2.586 25.86 0.0033600000000000075
[13.86  6.76  2.66  1.08  0.78  0.44  0.    0.08  0.16  0.04]
[0.9999 0.9999 0.9999 0.9999 0.9999 0.9999 0.9961 0.9899 0.9799 0.9879]
15 0.044 0.44 0.00023999999999997358
[0.   0.   0.   0.02 0.06 0.02 0.02 0.02 0.18 0.12]
[1.     1.     1.     1.     0.9998 0.9996 0.9964 0.9896 0.9794 0.9874]
20 2.468 24.68 0.00014399999999996637
[14.    6.62  2.68  1.04  0.02  0.02  0.04  0.02  0.22  0.02]
[0.99999 0.99999 0.99999 0.99999 0.99981 0.99941 0.99679 0.98961 0.97899
 0.98701]
25 0.026 0.26 2.3999999999890774e-05
[0.   0.   0.02 0.   0.02 0.02 0.   0.04 0.12 0.04]
[1.      1.      1.      0.99996 0.99984 0.9994  0.9968  0.98958 0.97894
 0.98696]
30 2.352 23.52 1

In [56]:
res = np.zeros(K, dtype=int)
mat, g = build_rand_mat(size - 1, x_min, y_min, x_max, y_max)
omega = g.omega
V = g.V
for i in range(len(mat)):
    k = hash_function(int(mat[i, 5]), K, size) 
    pfa = compute_pfa(mat[i, 4] / omega, V , mat[i,5])
    print(pfa)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999911384875
0.9999999798208312
0.9999999700691523
0.9999999875998891
0.9999999047643218
0.9999998613794215
0.9999998451084481
0.9999999604144829
0.999999981745761
0.9999999021206116
0.9999998940045819
0.9999999082609087
0.9999999616894677
0.9999999688822109
0.9999991706630476
0.999999962033505
0.9999987211584747
0.9999999106375322
0.9999986363246771
0.9999997891130711
0.9999993742809271
0.9999998971524051
0.9999989742958838
0.9999992421055128
0.9999842277884835
0.9999939208290389
0.9998045073352191
nan


In [ ]:
K = 10
deltas = (1 - 1e-3) * np.ones(K)
deltas = 0.98 * np.ones(K)
eps = 1
conf_level = 0.01
update_ = 1e-4
Q = 50

prev_deltas = deltas.copy()

for i in range(5001):
    if i == 500:
        update_ = 1e-5
    deltas, res_q = update_deltas(deltas, eps, mat, K, x_min, y_min, x_max, y_max, update_, Q=Q)
    if i % 50 == 0:
        diff_delta = np.mean(np.abs(prev_deltas - deltas))
        prev_deltas = deltas.copy()
        print(i, np.mean(res_q), np.mean(np.sum(res_q, axis=0)), diff_delta)
    if i % 200 == 0:
        print(deltas)
print(deltas)

In [21]:
K = 20
deltas = (1 - 1e-2) * np.ones(K)
eps = 0.1
conf_level = 0.05
update_ = 1e-5
Q = 50

for i in range(1001):
    deltas, res_q = update_deltas(deltas, eps, mat, K, x_min, y_min, x_max, y_max, update_, Q=Q)
    if i % 50 == 0:
        print(i, np.mean(res_q), np.mean(np.sum(res_q, axis=0)))
print(deltas)

0 0.037 0.74
50 0.031 0.62
100 0.036 0.72
150 0.032 0.64
200 0.03 0.6
250 0.025 0.5
300 0.023 0.46
350 0.025 0.5
400 0.033 0.66
450 0.018 0.36
500 0.025 0.5
550 0.027 0.54


KeyboardInterrupt: 

In [33]:
((1 - 1e-2) - deltas[9]) / update_

-1000.0000000000008

In [48]:
deltas[2]

0.9999699999999979

In [16]:
np.sum(res_q, axis=0)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 2, 0, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 4])

In [11]:
s = np.maximum(np.std(res_q, axis=1), 1e-5)
m = np.mean(res_q, axis=1)
t.cdf((m - eps / K) / s * np.sqrt(Q - 1), Q-1) > conf_level

array([False, False,  True,  True, False,  True,  True,  True,  True,
       False])

In [25]:
deltas

array([0.98534, 0.98534, 0.98534, 0.98534, 0.9854 , 0.99328, 0.99466,
       0.99466, 0.99466, 0.98534])